data insertion query to RDBMS database

<!-- data insertion query to RDBMS -->

In [12]:
import random
from faker import Faker
import pymysql

fake = Faker()

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="8210",
    database="db_6",
    auth_plugin_map={
        'caching_sha2_password': 'sha256_password'
    }
)
cursor = conn.cursor()

def batch_commit(i, batch_size=100):
    if (i + 1) % batch_size == 0:
        conn.commit()

# 1. Insert Policy Types
policy_types = ['Life', 'Health', 'Auto', 'Home']
policy_type_ids = []
for i, pt in enumerate(policy_types):
    cursor.execute("""
        INSERT INTO PolicyTypes (Name, Description, CoverageDetails, BasePremium)
        VALUES (%s, %s, %s, %s)
    """, (pt, fake.text(max_nb_chars=30), fake.text(max_nb_chars=40), round(random.uniform(1000, 5000), 2)))
    policy_type_ids.append(cursor.lastrowid)
conn.commit()

# 2. Insert Agents
agent_ids = []
for i in range(100):
    cursor.execute("""
        INSERT INTO Agents (FirstName, LastName, Email, Phone, AgencyName, Region, HireDate)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        fake.first_name(), fake.last_name(), fake.email(),
        fake.phone_number()[:50],  # truncate phone to 50 chars max
        fake.company(), fake.state(), fake.date()
    ))
    agent_ids.append(cursor.lastrowid)
    batch_commit(i)

# 3. Insert Customers
customer_ids = []
for i in range(1000):
    cursor.execute("""
        INSERT INTO Customers (FirstName, LastName, DateOfBirth, Gender, Email, Phone, Address, City, State, ZipCode)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        fake.first_name(), fake.last_name(),
        fake.date_of_birth(minimum_age=18, maximum_age=85),
        random.choice(['M', 'F']),
        fake.unique.email(),
        fake.phone_number()[:50],  # truncate phone to 50 chars max
        fake.address(), fake.city(), fake.state(), fake.zipcode()
    ))
    customer_ids.append(cursor.lastrowid)
    batch_commit(i)

# 4. Insert Underwriters
underwriter_ids = []
for i in range(50):
    cursor.execute("""
        INSERT INTO Underwriters (FirstName, LastName, Email, Phone, Department)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        fake.first_name(), fake.last_name(),
        fake.email(), fake.phone_number()[:50], fake.job()
    ))
    underwriter_ids.append(cursor.lastrowid)
    batch_commit(i)

# 5. Insert Policies
policy_ids = []
for i in range(1000):
    start_date = fake.date_between(start_date='-2y', end_date='-1y')
    end_date = fake.date_between(start_date=start_date, end_date='+1y')
    cursor.execute("""
        INSERT INTO Policies (CustomerID, AgentID, PolicyTypeID, PolicyNumber, StartDate, EndDate, Status, TotalPremium)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        random.choice(customer_ids), random.choice(agent_ids), random.choice(policy_type_ids),
        fake.bothify("POL#######"),
        start_date,
        end_date,
        random.choice(['Active', 'Expired', 'Cancelled', 'Pending']),
        round(random.uniform(1000, 10000), 2)
    ))
    policy_ids.append(cursor.lastrowid)
    batch_commit(i)

# 6. Insert Payments
for i in range(2000):
    cursor.execute("""
        INSERT INTO Payments (PolicyID, PaymentDate, Amount, PaymentMethod, Status)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        random.choice(policy_ids),
        fake.date_between(start_date='-1y', end_date='today'),
        round(random.uniform(100, 1000), 2),
        random.choice(['Credit Card', 'Bank Transfer', 'Cash']),
        random.choice(['Completed', 'Pending', 'Failed'])
    ))
    batch_commit(i)

# 7. Insert Claims
for i in range(500):
    cursor.execute("""
        INSERT INTO Claims (PolicyID, ClaimDate, ClaimAmount, Status, Description)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        random.choice(policy_ids),
        fake.date_between(start_date='-1y', end_date='today'),
        round(random.uniform(1000, 25000), 2),
        random.choice(['Filed', 'Approved', 'Rejected', 'Paid', 'In Review']),
        fake.text(max_nb_chars=100)
    ))
    batch_commit(i)

# 8. Insert Policy Underwriting
for i in range(1000):
    cursor.execute("""
        INSERT INTO PolicyUnderwriting (PolicyID, UnderwriterID, AssessmentNotes, AssessmentDate, RiskLevel)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        random.choice(policy_ids), random.choice(underwriter_ids),
        fake.text(max_nb_chars=50), fake.date_between(start_date='-2y', end_date='today'),
        random.choice(['Low', 'Medium', 'High'])
    ))
    batch_commit(i)

# 9. Insert Vehicles
for i in range(500):
    cursor.execute("""
        INSERT INTO Vehicles (PolicyID, Make, Model, Year, VIN, LicensePlate)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        random.choice(policy_ids), fake.company(), fake.word(),
        random.randint(2000, 2023), fake.bothify("VIN#######"), fake.bothify("??##??")
    ))
    batch_commit(i)

# 10. Insert Properties
for i in range(500):
    cursor.execute("""
        INSERT INTO Properties (PolicyID, Address, PropertyType, YearBuilt, EstimatedValue)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        random.choice(policy_ids), fake.address(),
        random.choice(['Residential', 'Commercial', 'Land']),
        random.randint(1950, 2022), round(random.uniform(50000, 1000000), 2)
    ))
    batch_commit(i)

# 11. Insert Beneficiaries
for i in range(800):
    cursor.execute("""
        INSERT INTO Beneficiaries (PolicyID, FullName, Relationship, Percentage)
        VALUES (%s, %s, %s, %s)
    """, (
        random.choice(policy_ids), fake.name(), random.choice(['Spouse', 'Child', 'Parent', 'Sibling']),
        random.choice([100.0, 50.0])
    ))
    batch_commit(i)

# Final commit any remaining inserts
conn.commit()
cursor.close()
conn.close()

print("✅ Successfully inserted data into all insurance-related tables.")


✅ Successfully inserted data into all insurance-related tables.
